In [55]:
import redis
import json
import time

In [56]:
redisMaster = redis.Redis(host='localhost', port=40000, decode_responses=True, password='password')
redisStandalone = redis.Redis(host='localhost', port=30001, decode_responses=True)

In [57]:
redisMaster.set('key', 'value')
redisMaster.get('key')

'value'

In [58]:
redisStandalone.set('key', 'value')
redisStandalone.get('key')

'value'

In [59]:
def teardown():
    redisMaster.flushdb()
    redisStandalone.flushdb()

teardown()

data = json.load(open('./data/games.json'))

In [60]:
count = 0
for id, info in data.items():
    count += 1
    if count == 10:
        break
    print(id, info['name'])

20200 Galactic Bowling
655370 Train Bandit
1732930 Jolt Project
1355720 Henosis™
1139950 Two Weeks in Painland
1469160 Wartune Reborn
1659180 TD Worlds
1968760 Legend of Rome - The Wrath of Mars
1178150 MazM: Jekyll and Hyde


In [61]:
def set_strings(client: redis.Redis):
    for id, info in data.items():
        client.set('string_games:'+id, info['name'])

start = time.time()
set_strings(redisStandalone)
standalone = time.time() - start

start = time.time()
set_strings(redisMaster)
master = time.time() - start

teardown()

In [62]:
print(f"Strings: Standalone time: {standalone}s, Cluster time: {master}s")

Strings: Standalone time: 30.617791891098022s, Cluster time: 34.321990728378296s


In [63]:
def set_unordered_map(client: redis.Redis):
    for id, info in data.items():
        client.hset('games_map', id, info['name'])

start = time.time()
set_unordered_map(redisStandalone)
standalone = time.time() - start

start = time.time()
set_unordered_map(redisMaster)
master = time.time() - start

teardown()

In [64]:
print(f"Unordered map: standalone time: {standalone}s, Cluster time: {master}s")

Unordered map: standalone time: 32.000972032547s, Cluster time: 34.033629179000854s


In [65]:
def set_ordered_map(client: redis.Redis):
    for id, info in data.items():
        client.zadd('games_map_ordered', {id: float(info['price'])})

start = time.time()
set_ordered_map(redisStandalone)
standalone = time.time() - start

start = time.time()
set_ordered_map(redisMaster)
master = time.time() - start

teardown()

In [66]:
print(f"Ordered map: standalone time: {standalone}s, Cluster time: {master}s")

Ordered map: standalone time: 31.987530946731567s, Cluster time: 33.95587420463562s


In [67]:
def set_list(client: redis.Redis):
    for id, info in data.items():
        client.lpush('ids', id)

start = time.time()
set_list(redisStandalone)
standalone = time.time() - start

start = time.time()
set_list(redisMaster)
master = time.time() - start

teardown()

In [68]:
print(f"List: standalone time: {standalone}s, Cluster time: {master}s")

List: standalone time: 30.968968868255615s, Cluster time: 34.32690668106079s
